In [ ]:
import matplotlib.pyplot as plt
import ipywidgets
import astropy.io.fits as fits
import numpy as np
import galsim
import os.path
%matplotlib widget

In [ ]:
def get_zk(opd):
    xs = np.linspace(-1, 1, opd.shape[0])
    ys = np.linspace(-1, 1, opd.shape[1])
    xs, ys = np.meshgrid(xs, ys)
    w = ~opd.mask
    basis = galsim.zernike.zernikeBasis(22, xs[w], ys[w], R_inner=0.61)
    zk, *_ = np.linalg.lstsq(basis.T, opd[w], rcond=None)
    return zk

In [ ]:
def sub_ptt(opd):
    xs = np.linspace(-1, 1, opd.shape[0])
    ys = np.linspace(-1, 1, opd.shape[1])
    xs, ys = np.meshgrid(xs, ys)
    zk = get_zk(opd)
    opd -= galsim.zernike.Zernike(zk[:4], R_inner=0.61)(xs, ys)
    return opd

In [ ]:
def getdata(angles, subsys, field):
    za, ra = angles
    name = f"z{za}_r{ra}_{subsys}"

    wf_fn = os.path.join(
        "wfsim_simple_opds", 
        "fea", 
        name,
        f"opd_{name}_{field}.fits.gz"
    )
    wfsim_opd = fits.getdata(wf_fn)
    wfsim_opd = np.ma.masked_array(wfsim_opd, mask=(wfsim_opd==0))
    
    tsph_fn = os.path.join(
        "ts_phosim_opds",
        "fea",
        name,
        f"opd_{name}_{field}.fits.gz"
    )
    tsph_opd = fits.getdata(tsph_fn)
    tsph_opd = np.ma.masked_array(tsph_opd, mask=(tsph_opd==0))
    return sub_ptt(wfsim_opd)*1e3, sub_ptt(tsph_opd)*1e3

In [ ]:
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    import matplotlib.pyplot as plt
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

In [ ]:
@ipywidgets.interact(
    angles=ipywidgets.Dropdown(
        options=[
            ("0,0",(0, 0)), 
            ("45,0", (45,0)), 
            ("45,45", (45, 45)), 
            ("30,-30", (30, -30))],
    ),
    subsys=ipywidgets.Dropdown(
        options=["M1M3", "M2", "Cam", "All"]
    ),
    field=ipywidgets.BoundedIntText(value=0, min=0, max=34)
)
def f(angles, subsys, field):
    wfsim_opd, tsph_opd = getdata(angles, subsys, field)
    vmax = np.max(np.abs(tsph_opd))
    
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(10, 3), sharex=True, sharey=True)    
    colorbar(axes[0].imshow(wfsim_opd, vmin=-vmax, vmax=vmax, cmap='plasma'))
    colorbar(axes[1].imshow(tsph_opd, vmin=-vmax, vmax=vmax, cmap='plasma'))
    colorbar(axes[2].imshow(wfsim_opd - tsph_opd, vmin=-vmax/25, vmax=vmax/25, cmap='seismic'))
    axes[0].set_title(f"wfsim RMS = {np.std(wfsim_opd):6.1f} nm")
    axes[1].set_title(f"tsph RMS = {np.std(tsph_opd):6.1f} nm")
    axes[2].set_title(f"(wfsim - tsph) RMS = {np.std(wfsim_opd - tsph_opd):6.1f} nm")
    plt.tight_layout()
    plt.show()

In [ ]:
plt.close('all')